In [ ]:
!pip install zarr

In [ ]:
import os
import zarr
import matplotlib.pyplot as plt

In [16]:
def display_tree(root_dir, indent=""):
    print(indent + os.path.basename(root_dir) + "/")
    indent += "    "
    
    try:
        for item in os.listdir(root_dir):
            item_path = os.path.join(root_dir, item)
            if os.path.isdir(item_path):
                display_tree(item_path, indent)
            else:
                print(indent + item)
                
    except PermissionError:
        print(indent + "[Access Denied]")

In [18]:
dir0="./kaggle/input/czii-cryo-et-object-identification/train/overlay/ExperimentRuns/TS_5_4"
display_tree(dir0)


TS_5_4/
    Picks/
        ribosome.json
        virus-like-particle.json
        beta-galactosidase.json
        beta-amylase.json
        apo-ferritin.json
        thyroglobulin.json


In [19]:
paths=[]
for dirname, _, filenames in os.walk(dir0):
    for filename in filenames:
        paths+=[(os.path.join(dirname, filename))]
paths

['./kaggle/input/czii-cryo-et-object-identification/train/overlay/ExperimentRuns/TS_5_4/Picks/ribosome.json',
 './kaggle/input/czii-cryo-et-object-identification/train/overlay/ExperimentRuns/TS_5_4/Picks/virus-like-particle.json',
 './kaggle/input/czii-cryo-et-object-identification/train/overlay/ExperimentRuns/TS_5_4/Picks/beta-galactosidase.json',
 './kaggle/input/czii-cryo-et-object-identification/train/overlay/ExperimentRuns/TS_5_4/Picks/beta-amylase.json',
 './kaggle/input/czii-cryo-et-object-identification/train/overlay/ExperimentRuns/TS_5_4/Picks/apo-ferritin.json',
 './kaggle/input/czii-cryo-et-object-identification/train/overlay/ExperimentRuns/TS_5_4/Picks/thyroglobulin.json']

In [23]:
import json

def load_from_json(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        data = json.load(f)
    return data

from collections import defaultdict
cnt = defaultdict(int)
L=[]
for i,path in enumerate(paths):
    data=load_from_json(path)
    cnt[i]=data['pickable_object_name']
    X=[]
    Y=[]
    Z=[]
    Z2=[]
    for datai in data['points']:
        xyz=datai['location']
        X+=[xyz['x']]
        Y+=[xyz['y']]
        Z+=[xyz['z']]
        Z2+=[xyz['z']//170]
    L+=[(i,X,Y,Z,Z2)]

cnt

defaultdict(int,
            {0: 'ribosome',
             1: 'virus-like-particle',
             2: 'beta-galactosidase',
             3: 'beta-amylase',
             4: 'apo-ferritin',
             5: 'thyroglobulin'})

In [ ]:
fig = plt.figure(figsize=(13,13))
ax = fig.add_subplot(111, projection='3d')

for data in L:
    i, X, Y, Z, Z2 = data  
    scatter = ax.scatter(X, Y, Z, c=[i]*len(X), vmin=0, vmax=5, cmap='viridis', label=cnt[i]) 

ax.set_xlabel('X Label')
ax.set_ylabel('Y Label')
ax.set_zlabel('Z Label')
cbar = fig.colorbar(scatter, ax=ax)
cbar.set_label('Label')
ax.legend()
plt.show()

In [1]:
dir0='./kaggle/input/czii-cryo-et-object-identification/train/static/ExperimentRuns/TS_5_4/VoxelSpacing10.000'

In [ ]:


files=os.listdir(dir0)
ZARR=[]
for file in files:
    ZARR+=[os.path.join(dir0,file)]
ZARR

['./kaggle/input/czii-cryo-et-object-identification/train/static/ExperimentRuns/TS_5_4/VoxelSpacing10.000/isonetcorrected.zarr',
 './kaggle/input/czii-cryo-et-object-identification/train/static/ExperimentRuns/TS_5_4/VoxelSpacing10.000/wbp.zarr',
 './kaggle/input/czii-cryo-et-object-identification/train/static/ExperimentRuns/TS_5_4/VoxelSpacing10.000/denoised.zarr',
 './kaggle/input/czii-cryo-et-object-identification/train/static/ExperimentRuns/TS_5_4/VoxelSpacing10.000/ctfdeconvolved.zarr']

In [ ]:
num = 2
for j, dir in enumerate(ZARR[num:num+1]):
    print(ZARR[num].split('/')[-1])
    data = zarr.open(dir, mode='r')
    fig = plt.figure(figsize=(12, 20))

    for i in range(35):
        ax = plt.subplot(7, 5, i + 1)
        plt.axis('off')
        image = data[2][i]
        min_val, max_val = image.min(), image.max()
        simage = ((image - min_val) / (max_val - min_val) * 255).astype('uint8')
        plt.imshow(simage)
    plt.show()
